In [1]:
import pandas as pd

#Recipes and Ingredients: Simple Cookbook
df_recipe_simple = pd.read_csv("data\\recipe.csv")
df_ingredient_simple = pd.read_csv("data\\ingredient.csv")

In [2]:
folder_path = "data\\atlas-query-cookbook"

df_ingredient = pd.read_csv(folder_path + r"\Ingredient", sep='\t')
df_ingredient['category'] = df_ingredient['category'].str.strip()
df_nutrition = pd.read_csv(folder_path + r"\Nutrition", sep='\t')
df_quantity = pd.read_csv(folder_path + r"\Quantity", sep='\t')
df_quantity['unit'] = df_quantity['unit'].str.strip()
df_recipe = pd.read_csv(folder_path + r"\Recipe", sep=',')

In [3]:
df_ingredient = (df_ingredient.drop(columns=['plural'])
                 .rename(columns = {"name": "ingredient_name"}))
new_order = ['ingredient_id', 'ingredient_name', 'category']
df_ingredient = df_ingredient[new_order]
df_ingredient

,ingredient_id,ingredient_name,category
0,1,1% lowfat cottage cheese,dairy
1,6,1% lowfat milk,dairy
2,10,10-inch flour tortilla,Mexican products
3,11,100% bran cereal,cereals
4,12,2% lowfat milk,dairy
...,...,...,...
3342,4641,cooked oatmeal,cereals
3343,4642,instant coffee granules,hot beverages
3344,4643,long grain enriched white rice,grains
3345,4644,frozen grapefruit juice concentrate,frozen fruit


In [4]:
df_quantity_ingredient = pd.merge(df_quantity, df_ingredient, on='ingredient_id')
df_quantity_ingredient["qty"] = (df_quantity_ingredient["min_qty"] + df_quantity_ingredient["max_qty"]) / 2
df_quantity_ingredient = (df_quantity_ingredient
                          .drop(columns = ["quantity_id", "max_qty", "min_qty", "preparation", "optional"]))

new_order = ['recipe_id', 'ingredient_id', 'ingredient_name', 'category', 'unit', 'qty']
df_quantity_ingredient = df_quantity_ingredient[new_order]
df_quantity_ingredient

,recipe_id,ingredient_id,ingredient_name,category,unit,qty
0,214,1613,graham cracker crumbs,cookies/crackers,cup(s),2.00
1,214,3334,sugar,baking products,cup(s),0.25
2,214,2222,margarine or butter,dairy,cup(s),0.50
3,214,2797,raspberry juice,fruit juices,cup(s),0.25
4,214,3567,unflavored gelatin,gelatin,teaspoon(s),3.00
...,...,...,...,...,...,...
5087,1119,289,whole boneless turkey,meat/poultry,pound(s),6.00
5088,1119,3261,Sonoma County chardonnay wine,alcoholic beverages,milliliter(s),750.00
5089,1119,638,chicken broth,soups,cup(s),1.00
5090,1119,1420,fresh rosemary,fresh herbs,teaspoon(s),3.00


In [5]:
def replacer(df, qty_type, multiplier, new_qty='gram(s)'):
    df.loc[df.unit == qty_type, 'qty'] *= multiplier
    df['unit'] = df['unit'].replace({qty_type: new_qty})
    return df

In [6]:
lst_qty = df_quantity.unit.unique()

for qty_type in lst_qty:
    ounce = qty_type.split(" ")[0]
    try:
        ounce_int = int(ounce)
    except:
        continue
    replacer(df_quantity_ingredient, qty_type, ounce_int * 29)
    
replacer(df_quantity_ingredient, 'pound(s)', 455)
replacer(df_quantity_ingredient, 'cup(s)', 240)
replacer(df_quantity_ingredient, 'ounce(s)', 29)
replacer(df_quantity_ingredient, 'fl. ounce(s)', 29)
replacer(df_quantity_ingredient, 'pint(s)', 12*29)
replacer(df_quantity_ingredient, 'drop(s)', 0.05, 'milliliter(s)')
replacer(df_quantity_ingredient, 'stalk(s)', 1, 'whole')
replacer(df_quantity_ingredient, 'head(s)', 1, 'whole')
replacer(df_quantity_ingredient, 'loaf', 1, 'whole')
replacer(df_quantity_ingredient, 'pinch(es)', 0.125, 'teaspoon(s)')
replacer(df_quantity_ingredient, 'dash(es)', 0.125, 'teaspoon(s)')
replacer(df_quantity_ingredient, 'packet(s)', 1, 'package(s)')
replacer(df_quantity_ingredient, 'envelope(s)', 1, 'package(s)')
replacer(df_quantity_ingredient, 'teaspoon(s)', 5)
replacer(df_quantity_ingredient, 'tablespoon(s)', 15)
replacer(df_quantity_ingredient, 'stick(s)', 1, 'piece(s)')
replacer(df_quantity_ingredient, 'slice(s)', 1, 'piece(s)')
replacer(df_quantity_ingredient, 'clove(s)', 1, 'piece(s)')
replacer(df_quantity_ingredient, 'scoop(s)', 1, 'piece(s)')
replacer(df_quantity_ingredient, 'bunch(es)', 1, 'piece(s)')
replacer(df_quantity_ingredient, 'sprig(s)', 1, 'piece(s)')

df_quantity_ingredient.loc[(df_quantity_ingredient.unit == 'quart(s)') & (df_quantity_ingredient.category.isin(['soups', 'beverages'])), 'qty'] *= 946
df_quantity_ingredient.loc[(df_quantity_ingredient.unit == 'quart(s)') & (df_quantity_ingredient.category.isin(['soups', 'beverages'])), 'unit'] = 'gram(s)'
replacer(df_quantity_ingredient, 'quart(s)', 1, 'piece(s)')

df_quantity_ingredient.where(df_quantity_ingredient.unit == 'gram(s)').dropna()   

,recipe_id,ingredient_id,ingredient_name,category,unit,qty
0,214.0,1613.0,graham cracker crumbs,cookies/crackers,gram(s),480.0
1,214.0,3334.0,sugar,baking products,gram(s),60.0
2,214.0,2222.0,margarine or butter,dairy,gram(s),120.0
3,214.0,2797.0,raspberry juice,fruit juices,gram(s),60.0
4,214.0,3567.0,unflavored gelatin,gelatin,gram(s),15.0
...,...,...,...,...,...,...
5086,1119.0,3618.0,vegetable oil,oils and fats,gram(s),30.0
5087,1119.0,289.0,whole boneless turkey,meat/poultry,gram(s),2730.0
5089,1119.0,638.0,chicken broth,soups,gram(s),240.0
5090,1119.0,1420.0,fresh rosemary,fresh herbs,gram(s),15.0


In [7]:
df_recipe_nutrition = pd.merge(df_recipe, df_nutrition, on='recipe_id')
df_recipe_nutrition = (df_recipe_nutrition.drop(columns=["servings", "yield_unit", "yield_unit", "prep_min", "cook_min", "stnd_min",
                                                         "source", "intro", "directions"]))

df_quantity_agg = (df_quantity[["recipe_id", "ingredient_id"]]
                    .groupby("recipe_id")["ingredient_id"]
                    .agg(list)
                    .reset_index()
                    .rename(columns = {"ingredient_id": "ingredients"}))
df_quantity_agg["ingredients"] = df_quantity_agg["ingredients"].apply(lambda x: sorted(x))

df_total_recipe = pd.merge(df_recipe_nutrition, df_quantity_agg, on = "recipe_id")
df_total_recipe

,recipe_id,title,subtitle,protein,carbo,alcohol,total_fat,sat_fat,cholestrl,sodium,iron,vitamin_c,vitamin_a,fiber,pcnt_cal_carb,pcnt_cal_fat,pcnt_cal_prot,calories,ingredients
0,214,Raspberry Chiffon Pie,NaN,5.47,41.29,0.00,11.53,2.21,1.39,260.78,0.81,8.89,586.20,0.87,56.80,35.68,7.53,290.79,"[924, 1414, 1613, 2196, 2222, 2797, 3334, 3334..."
1,215,Apricot Yogurt Parfaits,NaN,5.70,23.75,1.93,1.08,0.58,3.48,46.17,0.57,13.02,2738.24,0.62,67.38,6.89,16.17,141.01,"[71, 76, 1286, 2019, 2196, 3567]"
2,216,Fresh Apricot Bavarian,NaN,4.90,26.88,0.00,1.10,0.58,3.46,41.79,0.37,6.13,1521.10,0.34,78.45,7.24,14.30,137.06,"[1286, 2196, 3334, 3567, 3649]"
3,217,Fresh Peaches,with Banana Cream Whip,1.77,18.17,0.00,0.21,0.06,0.00,14.01,0.19,8.79,478.09,0.69,88.98,2.35,8.67,81.70,"[792, 1402, 2019]"
4,218,Canned Cherry Crisp,NaN,1.38,36.63,0.00,5.47,3.46,10.36,50.22,0.66,0.16,229.16,1.05,72.81,24.46,2.73,201.23,"[30, 34, 351, 374, 614, 770, 1684, 2948]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838,1410,No-Bake Chocolate Cheesecake,NaN,6.50,28.81,0.00,45.24,24.49,103.78,173.07,1.36,0.59,1283.70,0.56,21.01,74.24,4.74,548.44,"[35, 249, 374, 374, 496, 782, 792, 921, 1755, ..."
839,1411,Mexican Cocoa Torte,NaN,3.24,38.67,0.00,35.89,14.67,65.37,238.75,1.07,0.30,701.14,0.09,31.53,65.83,2.64,490.67,"[738, 1755, 1767, 2627, 3118, 3329, 3334]"
840,1426,-Pancakes-,NaN,6.29,29.24,0.00,6.50,1.78,59.34,244.73,0.57,0.43,137.48,0.08,58.30,29.15,12.55,200.63,"[30, 126, 2327, 3021, 3334, 3595, 3618]"
841,1427,-Waffles-,NaN,6.57,30.14,0.00,13.74,2.85,60.38,173.88,0.59,0.51,147.61,0.10,44.57,45.71,9.72,270.50,"[30, 125, 1696, 2327, 3021, 3334, 3618]"


In [8]:
def recipe_ingredients(df, recipe_id):
    return df[df["recipe_id"] == recipe_id]

recipe_ingredients(df_quantity_ingredient, 214)
recipe_ingredients(df_quantity_ingredient, 1020)

,recipe_id,ingredient_id,ingredient_name,category,unit,qty
2947,1020,756,cling peach halves in syrup,canned/bottled fruit,gram(s),928.0
2948,1020,967,cucumber,fresh vegetables,gram(s),240.0
2949,1020,1648,green onion,fresh vegetables,gram(s),120.0
2950,1020,2442,olive oil,oils and fats,gram(s),30.0
2951,1020,2196,plain lowfat yogurt,dairy,gram(s),232.0
2952,1020,1573,garlic clove,fresh vegetables,piece(s),1.0
2953,1020,1334,fresh ginger root,fresh herbs,gram(s),15.0
2954,1020,2026,lemon pepper,spices and seasonings,gram(s),2.5
2955,1020,1029,dill,spices and seasonings,gram(s),2.5
2956,1020,804,cooked chicken,meat/poultry,gram(s),720.0
